## 로컬 환경에서 PDF 파일 RAG 검색하기 3단계 
### - 사용한 임베딩 모델 : jhgan/ko-sroberta-multitask
### - 사용한 LLM 모델 : llama3.2

__step1__
- PDF 문서 여러개 로드 (data 폴더에 있는 문서 전부 로드)
- 문서를 임베딩하여 csv 파일로 저장 (저장 경로 : csv 폴더)
- csv 파일을 FAISS 인덱싱 : 결과물이 인덱스로 나옴
- FAISS 인덱스를 파일로 만들어 디스크에 저장
- 저장한 인덱스를 사용하여 랭체인 프레임워크를 적용하여 검색

In [6]:
# 필요한 라이브러리 임포트
from langchain.document_loaders import DirectoryLoader,PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_core.embeddings import Embeddings  
from sentence_transformers import SentenceTransformer

import csv
import faiss
import numpy as np
import pandas as pd
import os

# 1. 문서 로드
# data 폴더 안에 있는 pdf 파일 전부 로드하기
loader = DirectoryLoader(
    'data',
    glob='*.pdf',
    loader_cls=PyMuPDFLoader
)
docs = loader.load()

# 2. 문서 분할
# 텍스트를 1000자 단위로 나눔 (chunk size), 각 청크 간 50자씩 겹치도록 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 3. 임베딩을 하기 위한 클래스 생성
class KoSentenceTransformerEmbeddings(Embeddings):
    # 임베딩 모델 초기화 
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)
    # 여러개의 문서를 임베딩하여 벡터 데이터 생성 , 벡터화 된 각 문서가 리스트 형태로 반환 
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_numpy=True).tolist()
    # 검색 쿼리를 벡터화 
    def embed_query(self, text):
        return self.model.encode([text], convert_to_numpy=True).tolist()[0]
    
# 3.1. 임베드 모델 로딩
embedding_model = KoSentenceTransformerEmbeddings("jhgan/ko-sroberta-multitask")

# 3.2 문서를 임베딩 하여 csv 파일로 저장하기 위한 함수 생성
def save_embeddings_to_csv(documents, embedding_model, file_path):
    os.makedirs(file_path, exist_ok=True)

    file_docs = {}
    for doc in documents:
        file_name = os.path.basename(doc.metadata['source']).replace('.pdf','')
        if file_name not in file_docs:
            file_docs[file_name] = []
        file_docs[file_name].append(doc)
    
    for file_name, docs in file_docs.items():
        full_path = os.path.join(file_path, f"{file_name}.csv")

        #  임베딩 
        embeddings = embedding_model.embed_documents([doc.page_content for doc in docs])
        #  임베딩 결과를 CSV 로 저장
        with open(full_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(["document", "embedding"])
            
            for doc, embedding in zip(docs, embeddings):
                writer.writerow([doc.page_content, embedding])
        
        print(f"임베딩 데이터가 {full_path} 파일에 저장되었습니다.")       
    
# 3.3 함수 실행 하여 CSV 파일 생성
save_embeddings_to_csv(split_documents, embedding_model, 'csv/')


임베딩 데이터가 csv/SPRI_AI_Brief_2023년12월호_F.csv 파일에 저장되었습니다.
임베딩 데이터가 csv/AI기반_인파분석플랫폼구축_제안서.csv 파일에 저장되었습니다.
임베딩 데이터가 csv/운영체제_중간과제물.csv 파일에 저장되었습니다.


In [8]:
# 4. 임베딩 데이터를 FAISS 인덱싱하는 과정
# 4.1. CSV 파일을 로드하는 함수 
def load_embeddings_from_csv(filepath):
    df = pd.read_csv(filepath)
    # 문자열을 numpy 배열로 변환 
    df["embedding"] = df["embedding"].apply(lambda x: np.fromstring(x[1:-1], sep=','))
    return df

# 4.2. FAISS 인덱스를 생성하는 함수 
def create_faiss_index(df):
    embedding_dim = len(df["embedding"].iloc[0])
    index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스 생성
    embeddings = np.vstack(df["embedding"].values).astype("float32")  # numpy 배열 변환
    index.add(embeddings)  # FAISS 인덱스에 벡터 추가
    return index, embedding_dim

# 4.3. FAISS 인덱스를 저장하는 함수 
def save_faiss_index(index, index_filepath):
    faiss.write_index(index, index_filepath)
    print(f"FAISS 인덱스가 {index_filepath} 파일로 저장되었습니다!")


# 4.4. FAISS 인덱스를 불러오는 함수 
def load_faiss_index(index_filepath):
    if os.path.exists(index_filepath):
        index = faiss.read_index(index_filepath)
        print(f"저장된 FAISS 인덱스를 {index_filepath}에서 불러왔습니다!")
        return index
    else:
        print(f"{index_filepath} 파일이 존재하지 않습니다.")
        return None
    
# 4.5. 함수 실행
if __name__ == "__main__":
    csv_filepath = "./csv/운영체제_중간과제물.csv"  # CSV 파일 경로
    index_filepath = "./faiss_index/faiss.index"  # 저장할 FAISS 인덱스 경로

    # CSV에서 임베딩 로드
    df_embeddings = load_embeddings_from_csv(csv_filepath)

    # FAISS 인덱스 생성
    faiss_index, embedding_dim = create_faiss_index(df_embeddings)

    # 인덱스를 파일로 저장
    os.makedirs(os.path.dirname(index_filepath), exist_ok=True)  # 폴더 없으면 생성
    save_faiss_index(faiss_index, index_filepath)

    # 저장된 인덱스 불러오기 테스트
    loaded_faiss_index = load_faiss_index(index_filepath)

FAISS 인덱스가 ./faiss_index/faiss.index 파일로 저장되었습니다!
저장된 FAISS 인덱스를 ./faiss_index/faiss.index에서 불러왔습니다!


In [14]:

# 5. 저장된 인덱스 파일 + csv 파일을 읽어서 검색을 하는 과정
# 답변을 자연어로 주기 위해서는 csv 파일도 같이 로드해야함
import faiss
import numpy as np
import pandas as pd

# 5.1. 저장된 FAISS 인덱스 로드하는 함수
def load_faiss_index(index_filepath):
    index = faiss.read_index(index_filepath)
    print(f"저장된 FAISS 인덱스를 {index_filepath}에서 불러왔습니다!")
    return index

# 5.2. CSV에서 문서 로드 (문서 원본 데이터 필요) 하는 함수 
def load_documents_from_csv(csv_filepath):
    df = pd.read_csv(csv_filepath)
    return df["document"].tolist()  # 문서 원본 텍스트 리스트 반환

# 5.3. FAISS에서 유사한 문서 검색하는 함수 
def search_faiss(index, query_embedding, documents, top_k=5):
    query_embedding = np.array([query_embedding], dtype=np.float32)  # FAISS 입력 형식 변환
    distances, indices = index.search(query_embedding, top_k)  # 가장 가까운 top_k개 검색
    
    results = []
    for i in range(top_k):
        idx = indices[0][i]
        results.append((documents[idx], distances[0][i]))  # (문서, 거리)
    
    return results


# 5.4. 함수 실행 
if __name__ == "__main__":
    index_filepath = "./faiss_index/faiss.index"  # 저장된 FAISS 인덱스 파일 경로
    csv_filepath = "./csv/운영체제_중간과제물.csv"  # 원본 문서가 저장된 CSV 파일 경로

    # FAISS 인덱스 로드
    faiss_index = load_faiss_index(index_filepath)

    # 문서 원본 데이터 로드
    documents = load_documents_from_csv(csv_filepath)

    # 검색어 입력 및 임베딩 변환
    query_text = "어디 학과의 공지사항입니까?"
    query_embedding = embedding_model.embed_query(query_text)

    # FAISS 검색 실행
    results = search_faiss(faiss_index, query_embedding, documents, top_k=5)

    # 검색 결과 출력
    print("\n🔍 검색 결과:")
    for rank, (doc, score) in enumerate(results, start=1):
        print(f"{rank}. 점수: {score:.4f}\n   문서: {doc}\n")


✅ 저장된 FAISS 인덱스를 ./faiss_index/faiss.index에서 불러왔습니다!

🔍 검색 결과:
1. 점수: 140.8967
   문서: 중간과제물 과제명
2024학년도 1학기
개설학과
컴퓨터과학과
교과목명
운영체제
개설학년
3
과제유형
공통형
[과제명]
1. 다음에 대해 답하시오. (15점)
(1) 프로세스의 다섯 가지 상태가 무엇인지 쓰고 각각을 설명하시오.
(2) 다음과 같은 상황에서 문서 작성 프로그램의 프로세스 상태가 어떻게 변화하는지 
구체적으로 설명하시오.
나는 어제 쓰던 보고서를 마무리하기 위해 우선 문서 작성 프로그램을 실행시켰다. 메뉴에서 파
일 열기를 찾아 작성하던 보고서 파일을 불러왔다. 작성해둔 보고서가 양이 많아 불러오는 시간
이 다소 소요되었다. 이후 보고서 작성을 마무리한 뒤 저장 버튼을 눌렀는데 역시 몇 초의 시간
이 지난 후에야 저장이 완료되었다. 이제 보고서 작업이 끝났기에 메뉴에서 종료 버튼을 찾아 
문서 작성 프로그램 창을 닫았다.
2. 프로세스별 도착시각과 필요한 CPU 사이클이 표와 같을 때, 다음에 대해 답하시오. 
단, 모든 답안은 근거(과정에 대한 설명, 계산식 등)가 함께 제시되어야 한다. (15점)
      
프로세스
A
B
C
D
E
도착시각
0
2
5
6
7
CPU 사이클
4
3
1
5
2
(1) SJF 스케줄링과 HRN 스케줄링 중 하나만 선택하여, 선택한 스케줄링 알고리즘에 의해 
프로세스들이 수행되는 순서를 구체적인 시각과 함께 표시하시오.
(2) (1)의 결과에 대해 각 프로세스의 반환시간을 구하고, 평균반환시간을 계산하시오.
(3) SRT 스케줄링과 RR 스케줄링(시간 할당량=3) 중 하나만 이용하여 프로세스들이 수행
되는 순서와 시각, 각 프로세스의 반환시간, 다섯 프로세스의 평균반환시간을 구하시오.
[과제작성 시 지시사항] : 작성서식, 분량, 제출방법, 보조파일 사용 여부 등 기술
 - 제출파일 종류: 한글, MS-Word 파일, 또는 텍스트 추출 가능한 PDF
 - 파일 용

In [ ]:
# 6. 프롬프트 생성 
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean,and make sure the answer ends with '입니다'.

#Context: 
{context}

#Question:
{question}

#Answer(Ensure the response ends with '입니다'):"""
)